## 歡迎使用麻婆大老分析
這是利用 osusearch 與 osu api v1 取得圖譜資料，並利用難度名稱找出GD等資料。  
是以圖譜難度為單位去整理及統計麻婆的數據。  


### 輸入麻婆名稱與ID

In [1]:
mapper_id = '9590557'   # 輸入你的使用者id , 可以從你的profile網址後面的數字得知 : https://osu.ppy.sh/users/6984103
mapper_name_list = ['Firika','Oiso']     # 輸入當前帳號、過去改名帳號、GD難度名稱所使用的名稱

###  呼叫資料 (要稍微等待一下)

In [2]:
import requests
import pandas as pd
import datetime

def parse_date(td):
    resYear = float(td.days)/364.0
    resMonth = int((resYear - int(resYear))*364/30)
    resYear = int(resYear)
    resDay = int(td.days-(364*resYear+30*resMonth))
    return str(resYear) + "年" + str(resMonth) + "個月又" + str(resDay) + "天"

# osusearch
df_mapper = pd.DataFrame()
for mapper_name in mapper_name_list:
    page=0
    while True:
        r = requests.get('https://osusearch.com/query/?mapper='+mapper_name+'&offset='+str(page)).json()
        page+=1
        if len(r.get('beatmaps'))==0:
            break
        for i in r.get('beatmaps'):
            df_map = pd.DataFrame(i, index=[0])
            df_mapper = df_mapper.append(df_map)
df_mapper = df_mapper.drop_duplicates('beatmapset_id')
df_GD = pd.DataFrame({'mapper':mapper_name_list})
df_GD['GD'] = 0
df_mapper = pd.merge(df_mapper, df_GD, how='left', on=['mapper'])
df_mapper = df_mapper.fillna(1)  
df_mapper['total_length_class'] = df_mapper['total_length'].apply(lambda x:'< 1:39'      if x<99           else(
                                                                           '1:39 ~ 3:29' if x>=99 and x<209 else(
                                                                           '3:29 ~ 5:00' if x>=209 and x<300 else '> 5:00')))
df_mapper['difficulty_rating'] = df_mapper['difficulty'].apply(lambda x:'Easy'   if x<2              else(
                                                                        'Normal' if x>=2   and x<2.7 else(
                                                                        'Hard'   if x>=2.7 and x<4   else(
                                                                        'Insane' if x>=4   and x<5.3 else(
                                                                        'Expert' if x>=5.3 and x<6.5 else'Expert+')))))
df_mapper = df_mapper[['beatmapset_id','title','artist','mapper','GD','difficulty_name','difficulty_rating','language','genre','total_length_class','beatmap_status','play_count','favorites']]


# osu api v1
get_beatmaps = requests.get('https://osu.ppy.sh/api/get_beatmaps?k=13a36d70fd32e2f87fd2a7a89e4f52d54ab337a1&u='+mapper_id).json()
df_api = pd.DataFrame()
for i in get_beatmaps:
    beatmap = pd.DataFrame.from_dict(i, orient='index').T
    df_api = df_api.append(beatmap)
df_api['difficulty_rating'] = df_api['difficultyrating'].apply(lambda x:'Easy'   if float(x)<2              else(
                                                                        'Normal' if float(x)>=2   and float(x)<2.7 else(
                                                                        'Hard'   if float(x)>=2.7 and float(x)<4   else(
                                                                        'Insane' if float(x)>=4   and float(x)<5.3 else(
                                                                        'Expert' if float(x)>=5.3 and float(x)<6.5 else'Expert+')))))
df_api['GDer'] = df_api['version'].apply(lambda x: 1 if "'s" in x else 0) 
df_api = df_api[['artist','title','artist_unicode','title_unicode','approved','version','GDer','submit_date','last_update','approved_date','difficulty_rating','playcount']]
df_api['submit_date'] = pd.to_datetime(df_api['submit_date'], format='%Y-%m-%d %H:%M:%S')
df_api['last_update'] = pd.to_datetime(df_api['last_update'], format='%Y-%m-%d %H:%M:%S')
df_api['approved_date'] = pd.to_datetime(df_api['approved_date'], format='%Y-%m-%d %H:%M:%S')
df_api['mapping_spend_time'] = df_api['last_update']-df_api['submit_date']
df_api = df_api.astype({'playcount':'int'})

print('呼叫完成，請繼續執行下方程式')


呼叫完成，請繼續執行下方程式


### GD 列表

In [3]:
pd.set_option('display.max_rows', 1000)
df_GD_check = df_mapper.loc[df_mapper.GD==1].sort_values(by='difficulty_name', ascending=True).reset_index()
df_GD_check[['beatmapset_id','title','artist','mapper','difficulty_name']]

,beatmapset_id,title,artist,mapper,difficulty_name
0,961362,Horangpungnyuga,Narae,-ClariS-,Firika's Advanced
1,811948,Extra Mode,USAO,Otosaka-Yu,Firika's Another
2,1094459,Kessen,Atsushi,Yusomi,Firika's Another
3,707164,ouroVoros,Negentropy (a.k.a. Team Grimoire),Suzuki_1112,Firika's Another
4,750316,SAY BAY,P*Light,yf_bmp,Firika's Expert (Winner!)
5,911992,Necro Fantasia,MISATO,-Doge-,Firika's Extra
6,796766,Empyrean,himmel,Imouto koko,Firika's Extra
7,1207987,Toki toshite Violence (TV Size),halca,Yorita Yoshino,Firika's Extra
8,902268,youthful beautiful,Uchida Maaya,Hanazawa Kana,Firika's Extra
9,928058,Akari no Arika,Kicco,Dored,Firika's Extra


### 手動刪除搜尋歪掉的GD

In [4]:
drop_list = [64,65]  # 輸入上方GD列表中的數字編號來刪除搜尋歪掉的圖。  若無要刪除的，則中括號內不用輸入任何數字。 ( drop_list = [] )

### 整理

In [5]:
# 刪除歪掉的GD
drop_index_list = df_GD_check[df_GD_check.index.isin(drop_list)]['index'].to_list()
df_mapper = df_mapper.drop(df_mapper.index[drop_index_list])

# 做圖年齡
mapping_age = parse_date(datetime.datetime.now() - df_api.submit_date.min())

# mapset坑數
mapset_count = len(df_mapper.loc[df_mapper.GD==0])
# Rank數
rank_mapset_count = len(df_mapper.loc[(df_mapper.GD==0) & (df_mapper.beatmap_status==1)])
# 累積favorites (自身mapset)
favorites_count = df_mapper.loc[df_mapper.GD==0].favorites.sum()
# GD數
GD_count = len(df_mapper.loc[df_mapper.GD==1])
# Rank GD數
rank_GD_count = len(df_mapper.loc[(df_mapper.GD==1) & (df_mapper.beatmap_status==1)])

# 累積playcount
GD_playcount = df_mapper.loc[df_mapper.GD==1].play_count.sum()
self_platcount = df_api.loc[df_api.GDer==0].playcount.sum()
platcount_count = GD_playcount+self_platcount

# 第一張圖 與 上傳時間
df_first_mapset = df_api.sort_values(by='submit_date', ascending=True).head(1)
#first_mapset = df_first_mapset.artist_unicode[0]+' - '+df_first_mapset.title_unicode[0]
first_mapset = df_first_mapset.artist[0]+' - '+df_first_mapset.title[0]
first_mapset_submit_date = datetime.datetime.strftime(df_api.submit_date.min(), '%Y-%m-%d')

# 第一張Rank圖 與 上榜時間
df_first_rankmap = df_api.loc[df_api.approved=='1'].sort_values(by='approved_date', ascending=True).head(1)
#first_rankmap = df_first_rankmap.artist_unicode[0]+' - '+df_first_rankmap.title_unicode[0]
first_rankmap = df_first_rankmap.artist[0]+' - '+df_first_rankmap.title[0]
first_rank_date = datetime.datetime.strftime(df_api.loc[df_api.approved=='1'].approved_date.min(), '%Y-%m-%d')
# 開始作圖後多久rank
first_rank_spend_time = parse_date(df_api.loc[df_api.approved=='1'].approved_date.min() - df_api.submit_date.min())

# 花費最多時間做的mapset 與 花費時間
df_most_spend_map = df_api.sort_values(by='mapping_spend_time', ascending=False).head(1)
#most_spend_map = df_most_spend_map.artist_unicode[0]+' - '+df_most_spend_map.title_unicode[0]
most_spend_map = df_most_spend_map.artist[0]+' - '+df_most_spend_map.title[0]
most_spend_map_mapping_time = parse_date(df_most_spend_map.mapping_spend_time[0])

# 語言統計
#df_language =  pd.DataFrame(df_mapper.language.value_counts(normalize=True).round(2).astype(str))
df_language =  pd.DataFrame(df_mapper.language.value_counts())
# 類型統計
df_genre =  pd.DataFrame(df_mapper.genre.value_counts())
# 圖長統計
df_length_class = pd.DataFrame(df_mapper.total_length_class.value_counts())
# 難度統計
df_GD_rating = df_mapper.loc[df_mapper.GD==1].difficulty_rating
df_rating = df_api.loc[df_api.GDer==0].difficulty_rating
df_main_rating = df_GD_rating.append(df_rating)
df_difficulty_rating = pd.DataFrame(df_main_rating.value_counts())

print('_________________________'+mapper_name_list[0]+'_________________________ '+datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d'))
print('')
print('做圖年齡 : '+mapping_age)
print('')
print('開坑數量 : '+str(mapset_count))
print('Rank set : '+str(rank_mapset_count))
print('Favorites : '+str(favorites_count))
print('')
print('GD 數 : '+str(GD_count))
print('Rank GD : '+str(rank_GD_count))
print('')
print('作圖數量 : '+str(len(df_main_rating)))
print('Playcount : '+str(platcount_count))
print('')
print('_________________________ History _________________________')
print('')
print('第一張圖: '+first_mapset+'  (第一次上傳時間: '+first_mapset_submit_date+')')
print('')
print('第一張Rank圖: '+first_rankmap+'  (上榜時間: '+first_rank_date+')')
print('   開始作圖後花了 '+first_rank_spend_time+' Rank了第一張圖')
print('')
print('花最多時間mapping的圖: '+str(most_spend_map)+'  (花費時間: '+most_spend_map_mapping_time+')')
print('')
print('__________________________ Style __________________________')
print('')
print('難度統計:')
print(df_difficulty_rating.rename(columns={'difficulty_rating': ''}))
print('')
print('語言統計:')
print(df_language.rename(columns={'language': ''}))
print('')
print('類型統計:')
print(df_genre.rename(columns={'genre': ''}))
print('')
print('圖長統計:')
print(df_length_class.rename(columns={'total_length_class': ''}))

_________________________Firika_________________________ 2020-12-16

做圖年齡 : 3年6個月又5天

開坑數量 : 36
Rank set : 8
Favorites : 1417

GD 數 : 72
Rank GD : 41

作圖數量 : 147
Playcount : 2339057

_________________________ History _________________________

第一張圖: Beyond - Hui Se Gui Ji  (第一次上傳時間: 2017-06-18)

第一張Rank圖: sana - Kanojo wa Tabi ni Deru  (上榜時間: 2019-08-16)
   開始作圖後花了 2年2個月又1天 Rank了第一張圖

花最多時間mapping的圖: Beyond - Hui Se Gui Ji  (花費時間: 1年10個月又10天)

__________________________ Style __________________________

難度統計:
           
Insane   46
Hard     37
Normal   22
Expert   22
Easy     16
Expert+   4

語言統計:
                
Japanese      47
Other         42
Instrumental  13
English        3
Chinese        2
Korean         1

類型統計:
               
Unspecified  41
Video Game   19
Anime        18
Pop          13
Electronic   10
Rock          3
Novelty       2
Other         2

圖長統計:
               
1:39 ~ 3:29  48
3:29 ~ 5:00  28
< 1:39       25
> 5:00        7
